# Example Notebook

In [1]:
!ls /data/common/trade_data

 import_data_2017_enhanced.parq
 import_data_2018_enhanced.parq
 import_data_2019_enhanced.parq
 import_data_2020_enhanced.parq
'manifestDB - CMDT Codes directory.xlsx'
'manifestDB - Data Fields Description.xlsx'
'manifestDB - Main Coverage Ports Directory.xlsx'
'manifestDB - Packing Unit Description.xlsx'
 raw_import_20152016.parq
 raw_import_2017.parq
 raw_import_2018.parq
 raw_import_2019.parq
 raw_import_2020.parq
 w210_raw_data.ipynb


In [2]:
import pandas as pd
import numpy as np
import re
import dask.dataframe as ddf
from math import nan
import panel as pn
import dask
import dask.dataframe as dd

from dask.diagnostics import ProgressBar
ProgressBar().register()

In [3]:
df_dataFields= pd.read_excel('manifestDB - Data Fields Description.xlsx', sheet_name='RenameColumns')
display(df_dataFields.head())
len(df_dataFields)

<ipython-input-3-bd4e24d952d9>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df_dataFields= pd.read_excel('manifestDB - Data Fields Description.xlsx', sheet_name='RenameColumns')


,ConciseColumnNames
0,BookingNumber
1,BOLNumber
2,Shipper
3,ShipperAddress
4,Consignee


39

### Load Updated Column Names

In [4]:
df_data_descriptions = pd.read_excel('manifestDB - Data Fields Description.xlsx', sheet_name='RenameColumns')
print(len(df_data_descriptions.ConciseColumnNames.values))
new_col_names = df_data_descriptions.ConciseColumnNames.values

39


<ipython-input-4-3bacef9a3f3c>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df_data_descriptions = pd.read_excel('manifestDB - Data Fields Description.xlsx', sheet_name='RenameColumns')


### Load Port Codes

In [5]:
port_latlong_df = pd.read_csv('all_imports_port_data.csv')
port_latlong_df.head()

,Port Code,Country,Port Name,Port Lat,Port Long
0,AEAJM,UAE,Ajman,25.403787,55.526284
1,AEAUH,UAE,ABU DHABI,24.453835,54.377401
2,AEJEA,UAE,Jebel Ali,25.028782,55.123823
3,AEKLF,UAE,KHOR FAKKAN,25.351163,56.322784
4,AESHJ,UAE,Sharjah,25.346150,55.421063


### Cleanup Functions

In [6]:
def rename_columns(df, col_names):
    """
    df = dataframe to rename columns
    col_names = new col names
    """
    print("Columns before:", df.columns)
    df_temp = df.drop('Unnamed: 0', axis=1)
    df_temp.columns = col_names
    return df_temp

In [15]:
def add_latitude_longitude(df, col_name):
    #print(len(df))
    df_temp = df.merge(port_latlong_df[['Port Code', 'Country', 'Port Lat', 'Port Long']], how='left', left_on=col_name, right_on='Port Code') 
    df_temp = df_temp.rename(columns={"Port Lat": col_name+"_Latitude", "Port Long": col_name+"_Longitude", "Country": col_name+"_Country"})
    df_temp = df_temp.drop('Port Code', axis=1)
    #print(len(df_temp))
    return df_temp

In [8]:
def extract_dates(df):
    df['SailingDate'] = dd.to_datetime(df['SailingDateOfVessel'])
    df['SailingDateYear'] = df['SailingDate'].dt.year
    df['SailingDateMonth'] = df['SailingDate'].dt.month
    return df

### Cleanup 2017 Data

In [69]:
filename = '/data/common/trade_data/raw_import_2017.parq'

In [70]:
df_2017 = rename_columns(dd.read_parquet(filename), new_col_names) 

Columns before: Index(['Unnamed: 0', ' Booking No ', ' B/L No ', ' Shipper ',
       ' Shipper Address ', ' Consignee ', ' Consignee Address ', ' TEU ',
       ' FEU ', ' Trunk VVD ', ' Vessel Name', ' POR ', ' POL(Booking) ',
       ' POD(Booking) ', ' DEL ', ' WGT(KGS) ', ' MEA(CBM) ', ' PKG ',
       ' PKG Unit', ' REP CMDT', ' CMDT Code', ' CMDT',
       ' Customs Description ', ' Freight Term', ' Forward Name ',
       ' Notify Name ', ' Notify Address ', ' Also Notify Name ',
       ' Export Name ', 'Trunk POL', ' Trunk POD',
       ' Booking Contact Phone Number ', ' Booking Contact E-Mail ',
       ' SI Contact Phone Number ', ' SI Contact E-Mail ', '  Sailing DT ',
       ' REV.WEEK ', ' Activity Teu ', ' Activity Feu ', ' Mty P/up CY '],
      dtype='object')


In [71]:
df_2017.head()

[########################################] | 100% Completed |  1.4s


,BookingNumber,BOLNumber,Shipper,ShipperAddress,Consignee,ConsigneeAddress,TwentyFeetContainerCount,FortyFeetContainerCount,TrunkVVD,CarrierVesselName,...,TrunkPOD,BookingPartyPhoneNumber,BookingPartyEmail,ShippingInstructorPhoneNumber,ShippingInstructorEmail,SailingDateOfVessel,ShipmentRevenueWeek,ActivityTEU,ActivityFEU,EmptyContainersPickUpLocation
0,BKK717427400,BKK717427400,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12,M FLOOR,SM TOWER\nPHAHOLYOTHIN ROAD,SAM...","TO THE ORDER OF BANK OF AMERICA,N.A.","450 B STREET,SUITE 430\nSAN DIEGO,CA 92101-890...",0.0,5.0,ANTY0003W,AL ABDALI,...,ATLSC,02-2980024159,piyaporn_sa@thaiunion.co.th,22980024,onuma_ap@thaiunion.co.th,2017-01-05 16:45:00,NaN,0,5,THLKGY1
1,BKK717427401,BKK717427401,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12,M FLOOR,SM TOWER\nPHAHOLYOTHIN ROAD,SAM...","TO THE ORDER OF BANK OF AMERICA,N.A.","450 B STREET,SUITE 430\nSAN DIEGO,CA 92101-890...",0.0,5.0,ANTY0003W,AL ABDALI,...,ATLSC,02-2980024159,piyaporn_sa@thaiunion.co.th,22980024,onuma_ap@thaiunion.co.th,2017-01-05 16:45:00,NaN,0,5,THLKGY1
2,BKK717427402,BKK717427402,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12,M FLOOR,SM TOWER\nPHAHOLYOTHIN ROAD,SAM...","TO THE ORDER OF BANK OF AMERICA,N.A.","450 B STREET,SUITE 430\nSAN DIEGO,CA 92101-890...",0.0,5.0,ANTY0003W,AL ABDALI,...,ATLSC,02-2980024159,piyaporn_sa@thaiunion.co.th,22980024,onuma_ap@thaiunion.co.th,2017-01-05 16:45:00,NaN,0,5,THLKGY1
3,BKK718896200,BKK718896200,"OKEANOS FOOD CO.,LTD.","103 NONSEE RD.,CHONGNONSEE,\nYANNAWA,BANGKOK 1...",TO ORDER OF BANK OF AMERICA N.A,"450 B STREET, SUITE 430\nSAN DIEGO,CA 92101 U....",0.0,1.0,CNAG0061E,COSCO AQABA,...,NYCSC,"02-295 1991 Ext. 258, 263",juthamard@pakfood.co.th; speeddeetrans_848@hot...,22951991360,ratchadaporn@pakfood.co.th,2017-01-05 16:45:00,NaN,0,1,THLKGY1
4,BKK716997200,BKK716997200W,"CHIANGMAI FROZEN FOODS PUBLIC CO.,LTD.","149/34 SOI ANGLO PLAZA,\nSURAWONGSE ROAD,BANGK...",NEW YORK MUTUAL TRADING INC.,"77 METRO WAY,SECAUCUS,NJ 07094\nTEL:(201)933-9...",0.0,1.0,COHA0022E,COSCO AQABA,...,NYCSC,6340061-2,wachiraporn@cmfrozen.com,26340061,wachiraporn@cmfrozen.com,2017-01-05 16:45:00,NaN,0,1,THLKGY1


In [72]:
df_2017.columns

Index(['BookingNumber', 'BOLNumber', 'Shipper', 'ShipperAddress', 'Consignee',
       'ConsigneeAddress', 'TwentyFeetContainerCount',
       'FortyFeetContainerCount', 'TrunkVVD', 'CarrierVesselName',
       'PlaceOfReceipt', 'PortOfLoading', 'PortOfDischarge',
       'FinalDeliveryPort', 'CargoWeightKgs', 'CargoVolumeCubicMeters',
       'NumberOfPackages', 'PackingUnit', 'CommodityRepCode', 'CommodityCode',
       'CommodityDescription', 'CustomsDescription', 'FreightTerm',
       'FreightForwarderName', 'NotifyPartyName', 'NotifyPartyAddress',
       'AlsoNotifyOtherName', 'ExportName', 'TrunkPOL', 'TrunkPOD',
       'BookingPartyPhoneNumber', 'BookingPartyEmail',
       'ShippingInstructorPhoneNumber', 'ShippingInstructorEmail',
       'SailingDateOfVessel', 'ShipmentRevenueWeek', 'ActivityTEU',
       'ActivityFEU', 'EmptyContainersPickUpLocation'],
      dtype='object')

In [73]:
df_2017 = add_latitude_longitude(df_2017, 'PlaceOfReceipt')

In [74]:
df_2017.head()

[########################################] | 100% Completed |  3.2s


,BookingNumber,BOLNumber,Shipper,ShipperAddress,Consignee,ConsigneeAddress,TwentyFeetContainerCount,FortyFeetContainerCount,TrunkVVD,CarrierVesselName,...,ShippingInstructorPhoneNumber,ShippingInstructorEmail,SailingDateOfVessel,ShipmentRevenueWeek,ActivityTEU,ActivityFEU,EmptyContainersPickUpLocation,PlaceOfReceipt_Country,PlaceOfReceipt_Latitude,PlaceOfReceipt_Longitude
0,BKK717427400,BKK717427400,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12,M FLOOR,SM TOWER\nPHAHOLYOTHIN ROAD,SAM...","TO THE ORDER OF BANK OF AMERICA,N.A.","450 B STREET,SUITE 430\nSAN DIEGO,CA 92101-890...",0.0,5.0,ANTY0003W,AL ABDALI,...,22980024,onuma_ap@thaiunion.co.th,2017-01-05 16:45:00,NaN,0,5,THLKGY1,Thailand,13.722378,100.752880
1,BKK717427401,BKK717427401,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12,M FLOOR,SM TOWER\nPHAHOLYOTHIN ROAD,SAM...","TO THE ORDER OF BANK OF AMERICA,N.A.","450 B STREET,SUITE 430\nSAN DIEGO,CA 92101-890...",0.0,5.0,ANTY0003W,AL ABDALI,...,22980024,onuma_ap@thaiunion.co.th,2017-01-05 16:45:00,NaN,0,5,THLKGY1,Thailand,13.722378,100.752880
2,BKK717427402,BKK717427402,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12,M FLOOR,SM TOWER\nPHAHOLYOTHIN ROAD,SAM...","TO THE ORDER OF BANK OF AMERICA,N.A.","450 B STREET,SUITE 430\nSAN DIEGO,CA 92101-890...",0.0,5.0,ANTY0003W,AL ABDALI,...,22980024,onuma_ap@thaiunion.co.th,2017-01-05 16:45:00,NaN,0,5,THLKGY1,Thailand,13.722378,100.752880
3,BKK718896200,BKK718896200,"OKEANOS FOOD CO.,LTD.","103 NONSEE RD.,CHONGNONSEE,\nYANNAWA,BANGKOK 1...",TO ORDER OF BANK OF AMERICA N.A,"450 B STREET, SUITE 430\nSAN DIEGO,CA 92101 U....",0.0,1.0,CNAG0061E,COSCO AQABA,...,22951991360,ratchadaporn@pakfood.co.th,2017-01-05 16:45:00,NaN,0,1,THLKGY1,Thailand,13.097055,100.915227
4,BKK716997200,BKK716997200W,"CHIANGMAI FROZEN FOODS PUBLIC CO.,LTD.","149/34 SOI ANGLO PLAZA,\nSURAWONGSE ROAD,BANGK...",NEW YORK MUTUAL TRADING INC.,"77 METRO WAY,SECAUCUS,NJ 07094\nTEL:(201)933-9...",0.0,1.0,COHA0022E,COSCO AQABA,...,26340061,wachiraporn@cmfrozen.com,2017-01-05 16:45:00,NaN,0,1,THLKGY1,Thailand,13.722378,100.752880


In [75]:
df_2017 = add_latitude_longitude(df_2017, 'PortOfLoading')

In [76]:
df_2017.head()

[########################################] | 100% Completed |  4.9s


,BookingNumber,BOLNumber,Shipper,ShipperAddress,Consignee,ConsigneeAddress,TwentyFeetContainerCount,FortyFeetContainerCount,TrunkVVD,CarrierVesselName,...,ShipmentRevenueWeek,ActivityTEU,ActivityFEU,EmptyContainersPickUpLocation,PlaceOfReceipt_Country,PlaceOfReceipt_Latitude,PlaceOfReceipt_Longitude,PortOfLoading_Country,PortOfLoading_Latitude,PortOfLoading_Longitude
0,BKK717427400,BKK717427400,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12,M FLOOR,SM TOWER\nPHAHOLYOTHIN ROAD,SAM...","TO THE ORDER OF BANK OF AMERICA,N.A.","450 B STREET,SUITE 430\nSAN DIEGO,CA 92101-890...",0.0,5.0,ANTY0003W,AL ABDALI,...,NaN,0,5,THLKGY1,Thailand,13.722378,100.752880,Thailand,13.097055,100.915227
1,BKK717427401,BKK717427401,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12,M FLOOR,SM TOWER\nPHAHOLYOTHIN ROAD,SAM...","TO THE ORDER OF BANK OF AMERICA,N.A.","450 B STREET,SUITE 430\nSAN DIEGO,CA 92101-890...",0.0,5.0,ANTY0003W,AL ABDALI,...,NaN,0,5,THLKGY1,Thailand,13.722378,100.752880,Thailand,13.097055,100.915227
2,BKK717427402,BKK717427402,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12,M FLOOR,SM TOWER\nPHAHOLYOTHIN ROAD,SAM...","TO THE ORDER OF BANK OF AMERICA,N.A.","450 B STREET,SUITE 430\nSAN DIEGO,CA 92101-890...",0.0,5.0,ANTY0003W,AL ABDALI,...,NaN,0,5,THLKGY1,Thailand,13.722378,100.752880,Thailand,13.097055,100.915227
3,BKK718896200,BKK718896200,"OKEANOS FOOD CO.,LTD.","103 NONSEE RD.,CHONGNONSEE,\nYANNAWA,BANGKOK 1...",TO ORDER OF BANK OF AMERICA N.A,"450 B STREET, SUITE 430\nSAN DIEGO,CA 92101 U....",0.0,1.0,CNAG0061E,COSCO AQABA,...,NaN,0,1,THLKGY1,Thailand,13.097055,100.915227,Thailand,13.097055,100.915227
4,BKK716997200,BKK716997200W,"CHIANGMAI FROZEN FOODS PUBLIC CO.,LTD.","149/34 SOI ANGLO PLAZA,\nSURAWONGSE ROAD,BANGK...",NEW YORK MUTUAL TRADING INC.,"77 METRO WAY,SECAUCUS,NJ 07094\nTEL:(201)933-9...",0.0,1.0,COHA0022E,COSCO AQABA,...,NaN,0,1,THLKGY1,Thailand,13.722378,100.752880,Thailand,13.097055,100.915227


In [77]:
df_2017 = add_latitude_longitude(df_2017, 'PortOfDischarge')

In [78]:
df_2017.head()

[########################################] | 100% Completed |  7.0s


,BookingNumber,BOLNumber,Shipper,ShipperAddress,Consignee,ConsigneeAddress,TwentyFeetContainerCount,FortyFeetContainerCount,TrunkVVD,CarrierVesselName,...,EmptyContainersPickUpLocation,PlaceOfReceipt_Country,PlaceOfReceipt_Latitude,PlaceOfReceipt_Longitude,PortOfLoading_Country,PortOfLoading_Latitude,PortOfLoading_Longitude,PortOfDischarge_Country,PortOfDischarge_Latitude,PortOfDischarge_Longitude
0,BKK717427400,BKK717427400,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12,M FLOOR,SM TOWER\nPHAHOLYOTHIN ROAD,SAM...","TO THE ORDER OF BANK OF AMERICA,N.A.","450 B STREET,SUITE 430\nSAN DIEGO,CA 92101-890...",0.0,5.0,ANTY0003W,AL ABDALI,...,THLKGY1,Thailand,13.722378,100.752880,Thailand,13.097055,100.915227,USA,32.080926,-81.091177
1,BKK717427401,BKK717427401,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12,M FLOOR,SM TOWER\nPHAHOLYOTHIN ROAD,SAM...","TO THE ORDER OF BANK OF AMERICA,N.A.","450 B STREET,SUITE 430\nSAN DIEGO,CA 92101-890...",0.0,5.0,ANTY0003W,AL ABDALI,...,THLKGY1,Thailand,13.722378,100.752880,Thailand,13.097055,100.915227,USA,32.080926,-81.091177
2,BKK717427402,BKK717427402,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12,M FLOOR,SM TOWER\nPHAHOLYOTHIN ROAD,SAM...","TO THE ORDER OF BANK OF AMERICA,N.A.","450 B STREET,SUITE 430\nSAN DIEGO,CA 92101-890...",0.0,5.0,ANTY0003W,AL ABDALI,...,THLKGY1,Thailand,13.722378,100.752880,Thailand,13.097055,100.915227,USA,32.080926,-81.091177
3,BKK718896200,BKK718896200,"OKEANOS FOOD CO.,LTD.","103 NONSEE RD.,CHONGNONSEE,\nYANNAWA,BANGKOK 1...",TO ORDER OF BANK OF AMERICA N.A,"450 B STREET, SUITE 430\nSAN DIEGO,CA 92101 U....",0.0,1.0,CNAG0061E,COSCO AQABA,...,THLKGY1,Thailand,13.097055,100.915227,Thailand,13.097055,100.915227,USA,40.712728,-74.006015
4,BKK716997200,BKK716997200W,"CHIANGMAI FROZEN FOODS PUBLIC CO.,LTD.","149/34 SOI ANGLO PLAZA,\nSURAWONGSE ROAD,BANGK...",NEW YORK MUTUAL TRADING INC.,"77 METRO WAY,SECAUCUS,NJ 07094\nTEL:(201)933-9...",0.0,1.0,COHA0022E,COSCO AQABA,...,THLKGY1,Thailand,13.722378,100.752880,Thailand,13.097055,100.915227,USA,40.712728,-74.006015


In [79]:
df_2017 = add_latitude_longitude(df_2017, 'FinalDeliveryPort')

In [80]:
df_2017.head()

[########################################] | 100% Completed |  9.3s


,BookingNumber,BOLNumber,Shipper,ShipperAddress,Consignee,ConsigneeAddress,TwentyFeetContainerCount,FortyFeetContainerCount,TrunkVVD,CarrierVesselName,...,PlaceOfReceipt_Longitude,PortOfLoading_Country,PortOfLoading_Latitude,PortOfLoading_Longitude,PortOfDischarge_Country,PortOfDischarge_Latitude,PortOfDischarge_Longitude,FinalDeliveryPort_Country,FinalDeliveryPort_Latitude,FinalDeliveryPort_Longitude
0,BKK717427400,BKK717427400,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12,M FLOOR,SM TOWER\nPHAHOLYOTHIN ROAD,SAM...","TO THE ORDER OF BANK OF AMERICA,N.A.","450 B STREET,SUITE 430\nSAN DIEGO,CA 92101-890...",0.0,5.0,ANTY0003W,AL ABDALI,...,100.752880,Thailand,13.097055,100.915227,USA,32.080926,-81.091177,USA,32.080926,-81.091177
1,BKK717427401,BKK717427401,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12,M FLOOR,SM TOWER\nPHAHOLYOTHIN ROAD,SAM...","TO THE ORDER OF BANK OF AMERICA,N.A.","450 B STREET,SUITE 430\nSAN DIEGO,CA 92101-890...",0.0,5.0,ANTY0003W,AL ABDALI,...,100.752880,Thailand,13.097055,100.915227,USA,32.080926,-81.091177,USA,32.080926,-81.091177
2,BKK717427402,BKK717427402,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12,M FLOOR,SM TOWER\nPHAHOLYOTHIN ROAD,SAM...","TO THE ORDER OF BANK OF AMERICA,N.A.","450 B STREET,SUITE 430\nSAN DIEGO,CA 92101-890...",0.0,5.0,ANTY0003W,AL ABDALI,...,100.752880,Thailand,13.097055,100.915227,USA,32.080926,-81.091177,USA,32.080926,-81.091177
3,BKK718896200,BKK718896200,"OKEANOS FOOD CO.,LTD.","103 NONSEE RD.,CHONGNONSEE,\nYANNAWA,BANGKOK 1...",TO ORDER OF BANK OF AMERICA N.A,"450 B STREET, SUITE 430\nSAN DIEGO,CA 92101 U....",0.0,1.0,CNAG0061E,COSCO AQABA,...,100.915227,Thailand,13.097055,100.915227,USA,40.712728,-74.006015,USA,38.718342,-95.081635
4,BKK716997200,BKK716997200W,"CHIANGMAI FROZEN FOODS PUBLIC CO.,LTD.","149/34 SOI ANGLO PLAZA,\nSURAWONGSE ROAD,BANGK...",NEW YORK MUTUAL TRADING INC.,"77 METRO WAY,SECAUCUS,NJ 07094\nTEL:(201)933-9...",0.0,1.0,COHA0022E,COSCO AQABA,...,100.752880,Thailand,13.097055,100.915227,USA,40.712728,-74.006015,USA,40.712728,-74.006015


In [81]:
df_2017 = extract_dates(df_2017)

In [82]:
df_2017.head()

[########################################] | 100% Completed |  9.5s


,BookingNumber,BOLNumber,Shipper,ShipperAddress,Consignee,ConsigneeAddress,TwentyFeetContainerCount,FortyFeetContainerCount,TrunkVVD,CarrierVesselName,...,PortOfLoading_Longitude,PortOfDischarge_Country,PortOfDischarge_Latitude,PortOfDischarge_Longitude,FinalDeliveryPort_Country,FinalDeliveryPort_Latitude,FinalDeliveryPort_Longitude,SailingDate,SailingDateYear,SailingDateMonth
0,BKK717427400,BKK717427400,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12,M FLOOR,SM TOWER\nPHAHOLYOTHIN ROAD,SAM...","TO THE ORDER OF BANK OF AMERICA,N.A.","450 B STREET,SUITE 430\nSAN DIEGO,CA 92101-890...",0.0,5.0,ANTY0003W,AL ABDALI,...,100.915227,USA,32.080926,-81.091177,USA,32.080926,-81.091177,2017-01-05 16:45:00,2017,1
1,BKK717427401,BKK717427401,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12,M FLOOR,SM TOWER\nPHAHOLYOTHIN ROAD,SAM...","TO THE ORDER OF BANK OF AMERICA,N.A.","450 B STREET,SUITE 430\nSAN DIEGO,CA 92101-890...",0.0,5.0,ANTY0003W,AL ABDALI,...,100.915227,USA,32.080926,-81.091177,USA,32.080926,-81.091177,2017-01-05 16:45:00,2017,1
2,BKK717427402,BKK717427402,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12,M FLOOR,SM TOWER\nPHAHOLYOTHIN ROAD,SAM...","TO THE ORDER OF BANK OF AMERICA,N.A.","450 B STREET,SUITE 430\nSAN DIEGO,CA 92101-890...",0.0,5.0,ANTY0003W,AL ABDALI,...,100.915227,USA,32.080926,-81.091177,USA,32.080926,-81.091177,2017-01-05 16:45:00,2017,1
3,BKK718896200,BKK718896200,"OKEANOS FOOD CO.,LTD.","103 NONSEE RD.,CHONGNONSEE,\nYANNAWA,BANGKOK 1...",TO ORDER OF BANK OF AMERICA N.A,"450 B STREET, SUITE 430\nSAN DIEGO,CA 92101 U....",0.0,1.0,CNAG0061E,COSCO AQABA,...,100.915227,USA,40.712728,-74.006015,USA,38.718342,-95.081635,2017-01-05 16:45:00,2017,1
4,BKK716997200,BKK716997200W,"CHIANGMAI FROZEN FOODS PUBLIC CO.,LTD.","149/34 SOI ANGLO PLAZA,\nSURAWONGSE ROAD,BANGK...",NEW YORK MUTUAL TRADING INC.,"77 METRO WAY,SECAUCUS,NJ 07094\nTEL:(201)933-9...",0.0,1.0,COHA0022E,COSCO AQABA,...,100.915227,USA,40.712728,-74.006015,USA,40.712728,-74.006015,2017-01-05 16:45:00,2017,1


In [83]:
df_2017.to_parquet('import_data_2017_enhanced.parq')

[########################################] | 100% Completed | 14.0s


### Cleanup 2018 Data

In [27]:
filename = '/data/common/trade_data/raw_import_2018.parq'

In [28]:
df_2018 = rename_columns(dd.read_parquet(filename), new_col_names) 

Columns before: Index(['Unnamed: 0', ' Booking No ', ' B/L No ', ' Shipper ',
       ' Shipper Address ', ' Consignee ', ' Consignee Address ', ' TEU ',
       ' FEU ', ' Trunk VVD ', ' Vessel Name', ' POR ', ' POL(Booking) ',
       ' POD(Booking) ', ' DEL ', ' WGT(KGS) ', ' MEA(CBM) ', ' PKG ',
       ' PKG Unit', ' REP CMDT', ' CMDT Code', ' CMDT',
       ' Customs Description ', ' Freight Term', ' Forward Name ',
       ' Notify Name ', ' Notify Address ', ' Also Notify Name ',
       ' Export Name ', ' Trunk POL', ' Trunk POD',
       ' Booking Contact Phone Number ', ' Booking Contact E-Mail ',
       ' SI Contact Phone Number ', ' SI Contact E-Mail ', '  Sailing DT ',
       ' REV.WEEK ', ' Activity Teu ', ' Activity Feu ', ' Mty P/up CY '],
      dtype='object')


In [29]:
df_2018 = add_latitude_longitude(df_2018, 'PlaceOfReceipt')

In [30]:
df_2018 = add_latitude_longitude(df_2018, 'PortOfLoading')

In [31]:
df_2018 = add_latitude_longitude(df_2018, 'PortOfDischarge')

In [32]:
df_2018 = add_latitude_longitude(df_2018, 'FinalDeliveryPort')

In [33]:
df_2018 = extract_dates(df_2018)

In [34]:
df_2018.head()

[########################################] | 100% Completed |  9.9s


,BookingNumber,BOLNumber,Shipper,ShipperAddress,Consignee,ConsigneeAddress,TwentyFeetContainerCount,FortyFeetContainerCount,TrunkVVD,CarrierVesselName,...,PortOfLoading_Longitude,PortOfDischarge_Country,PortOfDischarge_Latitude,PortOfDischarge_Longitude,FinalDeliveryPort_Country,FinalDeliveryPort_Latitude,FinalDeliveryPort_Longitude,SailingDate,SailingDateYear,SailingDateMonth
0,FOC808686800,FOC808686800W,"MAGUS INDUSTRY CO., LTD.","6F,#300,JUI-KUANG RD.,NEIHU DIST,TA\nIPEI,TAIW...",EUROMARKET DESIGNS INC.,"1250 TECHNY ROAD,\nNORTHBROOK, IL 60062 U.S.A....",1.0,0.0,ELRN0011E,YM EMINENCE,...,116.353041,USA,37.804456,-122.271356,USA,37.804456,-122.271356,2018-01-05 16:45:00,2018,1
1,BKK818265600,BKK818265600,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12 M FLOOR S.M. TOWER\nPHAHOLYOTHIN ROAD,S...",TO ORDER OF BANK OF AMERICA ATTN:JOANN REGINA,"450 B STREET, SUITE 430\nSAN DIEGO,CA 92101 U....",0.0,1.0,ELRN0011E,COSCO EXCELLENCE,...,100.915227,USA,40.712728,-74.006015,USA,38.718342,-95.081635,2018-01-05 16:45:00,2018,1
2,BKK818268200,BKK818268200,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12 M FLOOR S.M. TOWER\nPHAHOLYOTHIN ROAD,S...",TO ORDER OF BANK OF AMERICA ATTN:JOANN REGINA,"450 B STREET, SUITE 430\nSAN DIEGO,CA 92101 U....",0.0,1.0,ELRN0011E,COSCO EXCELLENCE,...,100.915227,USA,40.712728,-74.006015,USA,38.718342,-95.081635,2018-01-05 16:45:00,2018,1
3,BKK818268200,BKK818268200,THAI UNION FROZEN PRODUCTS PUBLIC COMPANY LIMITED,"979/12 M FLOOR S.M. TOWER\nPHAHOLYOTHIN ROAD,S...",TO ORDER OF BANK OF AMERICA ATTN:JOANN REGINA,"450 B STREET, SUITE 430\nSAN DIEGO,CA 92101 U....",0.0,1.0,ELRN0011E,COSCO EXCELLENCE,...,100.915227,USA,40.712728,-74.006015,USA,38.718342,-95.081635,2018-01-05 16:45:00,2018,1
4,BKK820689200,BKK820689200W,"THAI SAMSUNG ELECTRONICS CO.,LTD. 313 MOO 1,SR...","SUKHAPIBAN 8 RD., TAMBON BEUNG, \nAMPHUR SRIRA...",SAMSUNG ELECTRONIC AMERICA INC AMERICAN DIST...,4921 DIRECTORS PLACE SUITE 200 \nSAN DIEGO...,0.0,1.0,ATSH1502E,COSCO EUROPE,...,100.915227,USA,40.712728,-74.006015,USA,39.829556,-75.534368,2018-01-05 16:45:00,2018,1


In [35]:
df_2018.to_parquet('import_data_2018_enhanced.parq')

[########################################] | 100% Completed | 13.7s


### Cleanup 2019 Data

In [36]:
filename = '/data/common/trade_data/raw_import_2019.parq'

In [37]:
df_2019 = rename_columns(dd.read_parquet(filename), new_col_names) 

Columns before: Index(['Unnamed: 0', ' Booking No ', ' B/L No ', ' Shipper ',
       ' Shipper Address ', ' Consignee ', ' Consignee Address ', ' TEU ',
       ' FEU ', ' Trunk VVD ', ' Vessel Name', ' POR ', ' POL(Booking) ',
       ' POD(Booking) ', ' DEL ', ' WGT(KGS) ', ' MEA(CBM) ', ' PKG ',
       ' PKG Unit', ' REP CMDT', ' CMDT Code', ' CMDT',
       ' Customs Description ', ' Freight Term', ' Forward Name ',
       ' Notify Name ', ' Notify Address ', ' Also Notify Name ',
       ' Export Name ', 'Trunk POL', 'Trunk POD',
       ' Booking Contact Phone Number ', ' Booking Contact E-Mail ',
       ' SI Contact Phone Number ', ' SI Contact E-Mail ', '  Sailing DT ',
       ' REV.WEEK ', ' Activity Teu ', ' Activity Feu ', ' Mty P/up CY '],
      dtype='object')


In [38]:
df_2019 = add_latitude_longitude(df_2019, 'PlaceOfReceipt')

In [39]:
df_2019 = add_latitude_longitude(df_2019, 'PortOfLoading')

In [40]:
df_2019 = add_latitude_longitude(df_2019, 'PortOfDischarge')

In [41]:
df_2019 = add_latitude_longitude(df_2019, 'FinalDeliveryPort')

In [42]:
df_2019 = extract_dates(df_2019)

In [43]:
df_2019.head()

[########################################] | 100% Completed | 18.0s


,BookingNumber,BOLNumber,Shipper,ShipperAddress,Consignee,ConsigneeAddress,TwentyFeetContainerCount,FortyFeetContainerCount,TrunkVVD,CarrierVesselName,...,PortOfLoading_Longitude,PortOfDischarge_Country,PortOfDischarge_Latitude,PortOfDischarge_Longitude,FinalDeliveryPort_Country,FinalDeliveryPort_Latitude,FinalDeliveryPort_Longitude,SailingDate,SailingDateYear,SailingDateMonth
0,SHCR94597300,SHCR94597300,ORIENT EXPRESS CONTAINER COMPANY LIMITED.,"ROOM I, 16FL, NO.726 WEST YAN AN RD\n, HUAMIN ...",STAR ASIA INTERNATIONAL INC. 208 CHURCH STREET...,"3328, U.S.A. EMAIL:ATTACH@STAR-\nTRACK.COM TEL...",1.0,0.0,YSJU1205E,HMM DURBAN,...,121.469207,USA,32.787601,-79.940273,USA,32.787601,-79.940273,2019-01-01 03:00:00,2019,1
1,SHEG94501900,SHEG94501900W,"BARTHCO INTERNATIONAL INC(XIAMEN) CO.,LTD-SHAN...","RM 409/410,HUAWEN INTERNATIONAL\nBUILDING,\nNO...",OHL INTERNATIONAL 1800-E ASSOCIATES LN.,"CHARLOTTE, NC 28217\nTEL: 704-916-0466 FAX:\n...",0.0,2.0,YJDU1248E,HMM DURBAN,...,121.469207,USA,32.787601,-79.940273,USA,32.787601,-79.940273,2019-01-01 03:00:00,2019,1
2,SHEG94502900,SHEG94502900W,"BARTHCO INTERNATIONAL INC(XIAMEN) CO.,LTD-SHAN...","RM 409/410,HUAWEN INTERNATIONAL\nBUILDING,\nNO...",OHL INTERNATIONAL 1800-E ASSOCIATES LN.,"CHARLOTTE, NC 28217\nTEL: 704-916-0466 FAX:\n...",0.0,3.0,YJDU1248E,HMM DURBAN,...,121.469207,USA,32.787601,-79.940273,USA,32.787601,-79.940273,2019-01-01 03:00:00,2019,1
3,SHEG94523900,SHEG94523900W,"BARTHCO INTERNATIONAL INC(XIAMEN) CO.,LTD-SHAN...","RM 409/410,HUAWEN INTERNATIONAL\nBUILDING,\nNO...",OHL INTERNATIONAL 1800-E ASSOCIATES LN.,"CHARLOTTE, NC 28217\nTEL: 704-916-0466 FAX:\n...",1.0,0.0,YJDU1248E,HMM DURBAN,...,121.469207,USA,32.787601,-79.940273,USA,32.787601,-79.940273,2019-01-01 03:00:00,2019,1
4,SHEI96719100,SHEI96719100W,MORGAN HOME FASHIONS(HK) LTDUNIT 19 01.19/F PO...,"HANOI ROAD, TSIM SHA\nHONG KONG, HONG KONG ISL...","WAL-MART STORES, INC.",601 N WALTON BLVD BENTONVILLE AR 72\n716-0410\...,0.0,4.0,YJDU1248E,HMM DURBAN,...,121.469207,USA,32.787601,-79.940273,USA,32.787601,-79.940273,2019-01-01 03:00:00,2019,1


In [44]:
df_2019.to_parquet('import_data_2019_enhanced.parq')

[########################################] | 100% Completed | 25.7s


### Cleanup 2020 Data

In [45]:
filename = '/data/common/trade_data/raw_import_2020.parq'

In [46]:
df_2020 = rename_columns(dd.read_parquet(filename), new_col_names) 

Columns before: Index(['Unnamed: 0', ' Booking No ', ' B/L No ', ' Shipper ',
       ' Shipper Address ', ' Consignee ', ' Consignee Address ', ' TEU ',
       ' FEU ', ' Trunk VVD ', ' Vessel Name', ' POR ', ' POL(Booking) ',
       ' POD(Booking) ', ' DEL ', ' WGT(KGS) ', ' MEA(CBM) ', ' PKG ',
       ' PKG Unit', ' REP CMDT', ' CMDT Code', ' CMDT',
       ' Customs Description ', ' Freight Term', ' Forward Name ',
       ' Notify Name ', ' Notify Address ', ' Also Notify Name ',
       ' Export Name ', '  Trunk POL', '  Trunk POD',
       ' Booking Contact Phone Number ', ' Booking Contact E-Mail ',
       ' SI Contact Phone Number ', ' SI Contact E-Mail ', '  Sailing DT ',
       ' REV.WEEK ', ' Activity Teu ', ' Activity Feu ', ' Mty P/up CY '],
      dtype='object')


In [47]:
df_2020 = add_latitude_longitude(df_2020, 'PlaceOfReceipt')

In [48]:
df_2020 = add_latitude_longitude(df_2020, 'PortOfLoading')

In [49]:
df_2020 = add_latitude_longitude(df_2020, 'PortOfDischarge')

In [50]:
df_2020 = add_latitude_longitude(df_2020, 'FinalDeliveryPort')

In [51]:
df_2020 = extract_dates(df_2020)

In [52]:
df_2020.head()

[########################################] | 100% Completed | 18.5s


,BookingNumber,BOLNumber,Shipper,ShipperAddress,Consignee,ConsigneeAddress,TwentyFeetContainerCount,FortyFeetContainerCount,TrunkVVD,CarrierVesselName,...,PortOfLoading_Longitude,PortOfDischarge_Country,PortOfDischarge_Latitude,PortOfDischarge_Longitude,FinalDeliveryPort_Country,FinalDeliveryPort_Latitude,FinalDeliveryPort_Longitude,SailingDate,SailingDateYear,SailingDateMonth
0,FOC202323700,FOC202323700W,"MAGUS INDUSTRY CO., LTD.","6F,#300,JUI-KUANG RD.,NEIHU DIST,TA\nIPEI,TAIW...",EUROMARKET DESIGNS INC.,"1250 TECHNY ROAD,\nNORTHBROOK, IL 60062 U.S.A....",1.0,0.0,CEUR1545E,YM MASCULINITY,...,116.353041,USA,37.804456,-122.271356,USA,37.804456,-122.271356,2020-01-01 03:00:00,2020,1
1,SZP2G9219200,SZP2G9219200,"OEC LOGISTICS (QINGDAO) CO., LTD. SHENZHEN B...","ROOM 12A01-02, NO.4018, ANLIAN \nPLAZA, JI...",OEC FREIGHT (NY) INC.,ONE CROSS ISLAND PLAZA 133-33 \nBROOKVILL...,0.0,5.0,CHLS1563E,XIN YA ZHOU,...,120.263043,USA,33.769016,-118.191604,USA,33.769016,-118.191604,2020-01-01 03:00:00,2020,1
2,TATK21239200,TATK21239200W,SAMSUNG ELECTRONICS (SHANDONG) DIGITAL PRINTIN...,(SSDP) SAMSUNG ROAD WEIHAI\nHI-TECH.IDZ WEIHAI...,SAMSUNG ELECTRONIC AMERICA AMERICAN DISTRIBUTI...,"4921 DIRECTORS PLACE SUITE 200\nSAN DIEGO, CA ...",0.0,8.0,HYKT1555E,HYUNDAI MARSEILLES,...,122.070000,USA,33.769016,-118.191604,USA,38.706046,-88.094935,2020-01-01 03:00:00,2020,1
3,TATK21266100,TATK21266100W,SAMSUNG ELECTRONICS (SHANDONG) DIGITAL PRINTIN...,(SSDP) SAMSUNG ROAD WEIHAI\nHI-TECH.IDZ WEIHAI...,SAMSUNG ELECTRONICS AMERICA. AMERICAN DISTRIBU...,"12014 EASTGATE BLVD.,MT JULIET,\nTENNESSEE 371...",0.0,3.0,HYKT1555E,HYUNDAI MARSEILLES,...,122.070000,USA,33.769016,-118.191604,.,NaN,NaN,2020-01-01 03:00:00,2020,1
4,TATK21556600,TATK21556600W,SAMSUNG ELECTRONICS (SHANDONG) DIGITAL PRINTIN...,SAMSUNG ROAD WEIHAI\nHI-TECH.IDZ WEIHAI CITY \...,SAMSUNG ELECTRONIC AMERICA AMERICAN DISTRIBUTI...,"4921 DIRECTORS PLACE SUITE 200\nSAN DIEGO, CA ...",0.0,5.0,HYKT1555E,HYUNDAI MARSEILLES,...,122.070000,USA,33.769016,-118.191604,USA,34.065846,-117.648430,2020-01-01 03:00:00,2020,1


In [53]:
df_2020.to_parquet('import_data_2020_enhanced.parq')

[########################################] | 100% Completed | 25.2s
